# 台灣公司資料

* 抓取檔案來自[台灣公司資料](http://company.g0v.ronny.tw/)
* 欄位定義看[這邊](http://company.g0v.ronny.tw/index/columns)
* 資料下載位置在[這裡](http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/index.html)

In [ ]:
import requests
import pandas as pd 

In [ ]:
url = 'http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/files/'

In [ ]:
# http://ronnywang-twcompany.s3-website-ap-northeast-1.amazonaws.com/files/00000000.json.gz
file_indexes = ['00000000','10000000','20000000','30000000','40000000','50000000','60000000','70000000','80000000','90000000']
# file_name = file_indexes[0] + '.json.gz'
path = '../data/'

In [ ]:
url + file_indexes[0] + '.jsonl.gz'

In [ ]:
def download_gz(path=path,file_indexes=file_indexes):
    ## 大型公司資料 00000000.json.gz 
    for item in file_indexes:
        url_data = url + item + '.jsonl.gz'
        res = requests.get(url_data)
        storage_path = path + item + '.jsonl.gz'
        with open(storage_path,'wb') as handle:
            if not res.ok:
                print('oops, something wrong!')
            else:
                for block in res.iter_content(1024):
                    handle.write(block)
                print '{} is downloaded'.format(item+'.jsonl.gz')
    ## 小企業資料 business-00000000.json.gz
    for item in file_indexes:
        file_path = 'bussiness-' + item + '.jsonl.gz'
        url_data = url + file_path
        res = requests.get(url_data)
        storage_path = path + file_path        
        with open(storage_path,'wb') as handle:
            if not res.ok:
                print('oops, something wrong')
            else:
                for block in res.iter_content(1024):
                    handle.write(block)        
                print '{} is downloaded'.format(file_path)

In [ ]:
download_gz()

## 處理.gz檔案

In [2]:
import gzip
import json

In [3]:
data = []
with gzip.open('../data/00000000.jsonl.gz') as f:
    for line in f:
        data.append(line)

In [4]:
data0 = json.loads(data[0])
data1 = json.loads(data[1])
# data1

In [5]:
print '/'.join(data0.keys())

代表人姓名/所營事業資料/經理人名單/核准設立日期/公司所在地/資本總額(元)/停業日期(迄)/董監事名單/舊營業項目資料/停業日期(起)/登記機關/最後核准變更日期/公司狀況/id/公司名稱


## 存入SQL

In [6]:
import sys
sys.getdefaultencoding()

'ascii'

In [7]:
import pypyodbc
import datetime

In [12]:
conn = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=External;unicode_results=True")
cursor = conn.cursor()

## 整理table
### 公司法人資料

In [8]:
datetime.datetime.strptime('20161201','%Y%m%d')

datetime.datetime(2016, 12, 1, 0, 0)

In [9]:
def get_company_info_str(data):
    if data:
        temp_info = ''
        for e in data:
            for j in e:                
                temp_info += '/'+j
        return temp_info
    else:
        return None
print get_company_info_str(data0[u'所營事業資料'])
print get_company_info_str(data1[u'所營事業資料'])   


None
//機械開發工程之承攬//道路土木建築橋樑等之工程之承攬//土地開發規劃改良設計//社區國民住宅研究及設計//與前各項有關進築材料經銷及產製//有關前項業務之經營與投資


In [10]:
def get_date(data_date):
    if data_date:
        [month,day,year] = data_date.values()
        return datetime.datetime.strptime(str(year)+'-' + str(month)+'-' +str(day),'%Y-%m-%d')
    else:
        return None
# data0[u'停業日期(起)']

In [11]:
def get_company_infos(row):
    company_list = []
    company_data_dict = {}
    company_data_dict['id'] = str(row['id'])
    company_data_dict[u'公司狀況'] = row.get(u'公司狀況')
    company_data_dict[u'公司名稱'] = row.get(u'公司名稱')
    if row.get(u'資本總額(元)') in [None,u'']:
        company_data_dict[u'資本總額(元)'] = None
    else: 
        company_data_dict[u'資本總額(元)'] = int((row.get(u'資本總額(元)')).replace(',',''))
        
    company_data_dict[u'代表人姓名'] = row.get(u'代表人姓名')
    company_data_dict[u'公司所在地'] = row.get(u'公司所在地')
    company_data_dict[u'登記機關'] = row.get(u'登記機關')
    company_data_dict[u'核准設立日期'] = get_date(row.get(u'核准設立日期'))
    company_data_dict[u'最後核准變更日期'] = get_date(row.get(u'最後核准變更日期'))
    company_data_dict[u'所營事業資料'] = get_company_info_str(row.get(u'所營事業資料'))
    company_data_dict[u'停業日期(起)'] = get_date(row.get(u'停業日期(起)'))
    company_data_dict[u'停業日期(迄)'] = get_date(row.get(u'停業日期(迄)'))
    company_data_dict[u'舊營業項目資料'] = row.get(u'舊營業項目資料')
    company_list.append(company_data_dict)
    return company_list

In [11]:
data106 = json.loads(data[106])

In [13]:
# # def create_db(table,cursor):
sql_create = u"""
create table [公司法人資料]
(
    [ID] varchar(40) not null,
    [公司狀況] VARCHAR(10),
    [公司名稱] VARCHAR(100),
    [資本總額(元)] BIGINT,
    [代表人姓名] VARCHAR(50) ,
    [公司所在地] VARCHAR(200),
    [登記機關] VARCHAR(20),
    [核准設立日期] smalldatetime,
    [最後核准變更日期] smalldatetime,
    [所營事業資料] VARCHAR(3000),
    [停業日期(起)] smalldatetime,
    [停業日期(迄)] smalldatetime,
    [舊營業項目資料] VARCHAR(2000),
    constraint pk_id primary key (ID)
)
"""
cursor.execute(sql_create)
cursor.commit()

In [14]:
sql_create_manager = u'''
create table [公司經理人]
(
    [ID] varchar(40) not null,
    [姓名] VARCHAR(20),
    [到職日期] smalldatetime
    constraint pk_all primary key (ID,[姓名],[到職日期])
)
'''

sql_create_director = u'''create table [公司董監事]
(
	[ID] varchar(40) not null,
	[姓名] VARCHAR(20),
	[職稱] text,
	[所代表法人] VARCHAR(100),
	[出資額] BIGINT
	constraint uc_director UNIQUE(ID,[姓名])
)
'''
cursor.execute(sql_create_manager)
cursor.execute(sql_create_director)
cursor.commit()

In [73]:
def to_db(datasets,table,cursor):
    
    for index, row in enumerate(datasets):
        # row data        
        fields = ','.join([ '['+e+ ']' for e in row.keys()]) # 欄位        
        values = row.values() # 值
        num_quest = '?' + ',?' * (len(row.keys())-1)
        try:
            sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(
                table, fields, num_quest)
            
            cursor.execute(sql_insert, values)
            # print "row index : %d, success!" %index
        except pypyodbc.IntegrityError as e:
            print 'Data key Duplicate!! Forbiden'
        except pypyodbc.DataError as e:
            print('Data Error, can\'t handle!')
        except pypyodbc.ProgrammingError as e:
            print('Programming error')
        except TypeError:
            print('Type error!!')
    cursor.commit()

In [15]:
print get_company_infos(data106)[0].get(u'\u6240\u71df\u4e8b\u696d\u8cc7\u6599')

//１﹒可變電容器、線圈及超高頻調諧器、汽車用調諧器、微小馬達、極高頻調諧器、//　調頻調諧器、錄音頭、電磁開關、可變電容二極體調諧器、混合積體電路（微小//　馬達用）之製造全部內外銷。//２﹒電視機用調諧器零件、微小馬達用零件、錄音頭用零件、可變電阻之製造全部內//　外銷。//３﹒生產「集體電路用插座」「電路用多聯接頭」「電路用壓著端子成型接頭」「電//　路接頭用壓著端子成型接頭」「電話線端子成型接頭」五項產品全部內外銷。//４﹒生產「線圈用空蕊自動捲線機」「線圈用空蕊自動銲錫機」「電阻器整理機」「//　電晶體整理機」「環銲錫機」「可變電容器自動組立機」「微小馬達用組立機」//　等七項產品全部內外銷。//５﹒生產調變器全部內外銷。//６﹒生產「有線電視變頻器」「高頻開關」「鍵盤開關」「交換式電源供應器」全部//　內外銷。//７﹒生產人造衛星接收變頻器內外銷。//８﹒生產電視用紅外線遙控器內外銷。//９﹒生產「電視遊樂器」產品全部內外銷。//１０﹒生產照相機用閃光燈。//１１﹒一般進出口貿易或代理國內外廠商有關產品之經銷投標及報價（許可業務除外）//　。//１２﹒發貨中心保稅倉庫業務（其範圍以經海關核准者為限）。//１３﹒磁碟機之製造加工及進出口業務。//１４﹒警報監視（聽）器之製造加工及進出口業務。//１５﹒光碟機、捕捉卡、音效卡之製造加工及進出口買賣業務。//１６﹒機器控制基板之製造加工及進出口買賣業務。//１７﹒電氣回路保護用之基板組立之製造加工及進出口買賣業務。/CC01030/電器製造業/CC01050/資料儲存及處理設備製造業/CC01060/有線通信機械器材製造業/CC01070/無線通信機械器材製造業/CC01080/電子零組件製造業/E601020/電器安裝業/E605010/電腦設備安裝業/E701010/通信工程業/E701020/衛星電視ＫＵ頻道、Ｃ頻道器材安裝業/E701030/電信管制射頻器材裝設工程業/F113020/電器批發業/F113050/事務性機器設備批發業/F113070/電信器材批發業/F119010/電子材料批發業/F213010/電器零售業/F213030/事務性機器設備零售業/F213060/電信器材零售業/F219010/電子材料零售業


In [16]:
table = u'公司法人資料' 
to_db(get_company_infos(data106),table,cursor)

###  公司董監事資料

In [18]:
print '/'.join(data1[u'董監事名單'][0].keys())

序號/出資額/姓名/職稱/所代表法人


In [19]:
for e in data1[u'董監事名單']:
    print e

{u'\u5e8f\u865f': u'0000', u'\u51fa\u8cc7\u984d': u'1,000,000', u'\u59d3\u540d': u'\u738b\u632f\u6797', u'\u8077\u7a31': u'\u57f7\u884c\u696d\u52d9\u80a1\u6771', u'\u6240\u4ee3\u8868\u6cd5\u4eba': u''}


In [16]:
def get_director_lists(row):
    director_lists = []
    
    id = row['id']
    if row.get(u'董監事名單'):
        for e in row.get(u'董監事名單'):
            director = {}
            director['id'] = id
            director[u'姓名'] = e.get(u'姓名')
            director[u'職稱'] = e.get(u'職稱')
            director[u'出資額'] = int(e.get(u'出資額').replace(',',''))
            if e.get(u'所代表法人'):
                director[u'所代表法人'] = e.get(u'所代表法人')[1]
            else:
                director[u'所代表法人'] = None
            director_lists.append(director)
        return director_lists
    else:
        return []

In [20]:
for e in data106.get(u'董監事名單'):
    print e

{u'\u5e8f\u865f': u'0001', u'\u51fa\u8cc7\u984d': u'3,175,127', u'\u59d3\u540d': u'\u68ee\u90e8\u4e00\u592b', u'\u8077\u7a31': u'\u8463\u4e8b\u9577', u'\u6240\u4ee3\u8868\u6cd5\u4eba': [0, u'\u65e5\u5546\u4e09\u7f8e\u96fb\u6a5f\u682a\u5f0f\u6703\u793e']}
{u'\u5e8f\u865f': u'0002', u'\u51fa\u8cc7\u984d': u'3,175,127', u'\u59d3\u540d': u'\u6ff1\u4e95\u82f1\u654f', u'\u8077\u7a31': u'\u8463\u4e8b', u'\u6240\u4ee3\u8868\u6cd5\u4eba': [0, u'\u65e5\u5546\u4e09\u7f8e\u96fb\u6a5f\u682a\u5f0f\u6703\u793e']}
{u'\u5e8f\u865f': u'0003', u'\u51fa\u8cc7\u984d': u'3,175,127', u'\u59d3\u540d': u'\u83c5\u539f\u6b66', u'\u8077\u7a31': u'\u8463\u4e8b', u'\u6240\u4ee3\u8868\u6cd5\u4eba': [0, u'\u65e5\u5546\u4e09\u7f8e\u96fb\u6a5f\u682a\u5f0f\u6703\u793e']}
{u'\u5e8f\u865f': u'0004', u'\u51fa\u8cc7\u984d': u'3,175,127', u'\u59d3\u540d': u'\u65e5\u91ce \u96c4\u4e8c', u'\u8077\u7a31': u'\u76e3\u5bdf\u4eba', u'\u6240\u4ee3\u8868\u6cd5\u4eba': [0, u'\u65e5\u5546\u4e09\u7f8e\u96fb\u6a5f\u682a\u5f0f\u6703\u793e'

In [21]:
for k, v in get_director_lists(data106)[3].items():
    print u'{},{}'.format(k,v)

所代表法人,日商三美電機株式會社
職稱,監察人
id,69149
姓名,日野 雄二
出資額,3175127


In [27]:
to_db(get_director_lists(data1),u'公司董監事',cursor)

In [28]:
to_db(get_director_lists(data106),u'公司董監事',cursor)

### 公司經理人資料

In [21]:
data3 = json.loads(data[100])

In [22]:
print '/'.join(data3[u'經理人名單'][0].keys())

序號/到職日期/姓名


In [23]:
data3[u'經理人名單'][0].values()

[u'0001', {u'day': 25, u'month': 10, u'year': 2011}, u'\u6797\u6b63\u5e38']

In [17]:
def get_manager_lists(row):
    manager_lists = []
    manager = {}
    id = row['id']
    if row.get(u'經理人名單'):
        for e in row.get(u'經理人名單'):
            manager['id'] = id
            manager[u'姓名'] = e.get(u'姓名')
            manager[u'到職日期'] = get_date(e.get(u'到職日期'))
            manager_lists.append(manager)    
        return manager_lists
    
    else:
        return []

In [31]:
table = u'公司經理人'
data_manager_lists = get_manager_lists(data1)
to_db(data_manager_lists,table,cursor)

## 解析json資料

In [65]:
data2330 = json.loads(data[2330])
get_company_infos(data2330)

[{'id': '940067',
  u'\u4ee3\u8868\u4eba\u59d3\u540d': u'\u5468\u6167\u798e',
  u'\u505c\u696d\u65e5\u671f(\u8d77)': None,
  u'\u505c\u696d\u65e5\u671f(\u8fc4)': None,
  u'\u516c\u53f8\u540d\u7a31': u'\u6551\u4eba\u5be6\u696d\u80a1\u4efd\u6709\u9650\u516c\u53f8',
  u'\u516c\u53f8\u6240\u5728\u5730': u'\u81fa\u5317\u5e02\u842c\u83ef\u5340\u6d1b\u967d\u88579\u865f9\u6a13\u4e4b1',
  u'\u516c\u53f8\u72c0\u6cc1': u'\u6838\u51c6\u8a2d\u7acb',
  u'\u6240\u71df\u4e8b\u696d\u8cc7\u6599': u'/F108021/\u897f\u85e5\u6279\u767c\u696d/F208021/\u897f\u85e5\u96f6\u552e\u696d/F108011/\u4e2d\u85e5\u6279\u767c\u696d/F208011/\u4e2d\u85e5\u96f6\u552e\u696d/C802090/\u6e05\u6f54\u7528\u54c1\u88fd\u9020\u696d/F107030/\u6e05\u6f54\u7528\u54c1\u6279\u767c\u696d/F207030/\u6e05\u6f54\u7528\u54c1\u96f6\u552e\u696d/C802100/\u5316\u7ca7\u54c1\u88fd\u9020\u696d/F108040/\u5316\u7ca7\u54c1\u6279\u767c\u696d/F208040/\u5316\u7ca7\u54c1\u96f6\u552e\u696d/C103050/\u7f50\u982d\u3001\u51b7\u51cd\u3001\u812b\u6c34\u53ca\u9183\

In [66]:
to_db(get_company_infos(data2330),u'公司法人資料',cursor)

Data key Duplicate!! Forbiden


In [70]:
filepath = '../data/00000000.jsonl.gz'
def json_parser_tosql(filepath):
#     conn = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=External;")    
#     cursor = conn.cursor()
    
    with gzip.open(filepath) as f:
        index = 0         
        for line in f:
            try:
                json_data = json.loads(line)

                ## 公司法人資料
                company_list = get_company_infos(json_data)

                to_db(company_list,u'公司法人資料',cursor)
                ## 公司董監事
    #             director_list = get_director_lists(json_data)
                to_db(get_director_lists(json_data),u'公司董監事',cursor)
                ## 公司經理人
                manager_list = get_manager_lists(json_data)
                to_db(manager_list,u'公司經理人',cursor)

                index+=1 
#                 print('line {} finished...'.format(index))
            except ValueError as e:
                print('Value Error when loading .gz file, occur index: {}'.format(index))

In [71]:
to_db(get_director_lists(data1),u'公司董監事',cursor)

Data key Duplicate!! Forbiden


# RUN

In [58]:
conn = pypyodbc.connect("DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=External;")
cursor = conn.cursor()

In [ ]:
json_parser_tosql(filepath)

Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key Duplicate!! Forbiden
Data key D

In [ ]:
get_company_infos(filepath)

In [54]:
print u'\u516c\u53f8\u7d93\u7406\u4eba'

公司經理人


In [35]:
data209 = json.loads(data[209])
print get_company_infos(data209)
print get_director_lists(data209)
print get_manager_lists(data209)

[{u'\u4ee3\u8868\u4eba\u59d3\u540d': u'', u'\u6240\u71df\u4e8b\u696d\u8cc7\u6599': None, u'\u6838\u51c6\u8a2d\u7acb\u65e5\u671f': datetime.datetime(1972, 5, 24, 0, 0), u'\u516c\u53f8\u6240\u5728\u5730': u'\xa0\xa0', u'\u8cc7\u672c\u7e3d\u984d(\u5143)': None, u'\u505c\u696d\u65e5\u671f(\u8fc4)': None, u'\u820a\u71df\u696d\u9805\u76ee\u8cc7\u6599': u'http://gcis.nat.gov.tw../cmpy/cmpyOriginalBusiItemAction.do?method=detail&banNo=00146997', u'\u505c\u696d\u65e5\u671f(\u8d77)': None, u'\u767b\u8a18\u6a5f\u95dc': u'\u81fa\u5317\u5e02\u653f\u5e9c', u'\u6700\u5f8c\u6838\u51c6\u8b8a\u66f4\u65e5\u671f': None, u'\u516c\u53f8\u72c0\u6cc1': u'\u89e3\u6563\u5df2\u6e05\u7b97\u5b8c\u7d50', 'id': '146997', u'\u516c\u53f8\u540d\u7a31': u'\u91d1\u7076\u4f01\u696d\u6709\u9650\u516c\u53f8'}]
[]
[]


In [40]:
table = u'公司法人資料'
for index, row in enumerate(get_company_infos(data209)):
    # row data        
    fields = ','.join([ '['+e+ ']' for e in row.keys()]) # 欄位        
    values = row.values() # 值
    num_quest = '?' + ',?' * (len(row.keys())-1)

    sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(table, fields, num_quest)

        

In [43]:
print sql_insert
print values

INSERT INTO 公司法人資料([代表人姓名],[所營事業資料],[核准設立日期],[公司所在地],[資本總額(元)],[停業日期(迄)],[舊營業項目資料],[停業日期(起)],[登記機關],[最後核准變更日期],[公司狀況],[id],[公司名稱]) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)
[u'', None, datetime.datetime(1972, 5, 24, 0, 0), u'\xa0\xa0', None, None, u'http://gcis.nat.gov.tw../cmpy/cmpyOriginalBusiItemAction.do?method=detail&banNo=00146997', None, u'\u81fa\u5317\u5e02\u653f\u5e9c', None, u'\u89e3\u6563\u5df2\u6e05\u7b97\u5b8c\u7d50', '146997', u'\u91d1\u7076\u4f01\u696d\u6709\u9650\u516c\u53f8']


In [36]:
to_db(get_company_infos(data209),u'公司法人資料',cursor)

DataError: (u'22001', u'[22001] [Microsoft][ODBC SQL Server Driver][SQL Server]String or binary data would be truncated.')

In [ ]:
print data106[u'id']

In [ ]:
print u'\u6240\u71df\u4e8b\u696d\u8cc7\u6599'
print u'\u8077\u7a31' # data106 error .... ntext 

In [35]:
company_list = get_company_infos(data106)
director_list = get_director_lists(data106)
to_db(director_list,u'公司董監事',cursor)

ProgrammingError: (u'42000', u"[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Disallowed implicit conversion from data type ntext to data type varchar, table 'External.dbo.\u516c\u53f8\u8463\u76e3\u4e8b', column '\u8077\u7a31'. Use the CONVERT function to run this query.")

In [ ]:
director_list[0][u'職稱']

In [ ]:
for k,v in director_list[0].items():
    print u"{}:{}".format(k,v)
    

In [ ]:
print u'\u8077\u7a31'

In [ ]:
data105 = json.loads(data[105])

In [ ]:
data106[u'董監事名單'][0][u'職稱']

In [ ]:
for e in data106[u'董監事名單']:
    for k,v in e.items():
        print (u'{}:{}'.format(k,v))

In [ ]:
director_list = get_director_lists(data1)

In [ ]:
director_list_106 = get_director_lists(data106)

In [ ]:
for k,v in director_list[0].items():
    print u'{}:{}'.format(k,v)

In [ ]:
for k,v in director_list_106[0].items():
    print u'{}:{}'.format(k,v)

In [ ]:
row = director_list_106[4]
fields = ','.join([ '['+e+ ']' for e in row.keys()])
values = row.values()
num_quest = '?' + ',?' * (len(row.keys())-1)
sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(
            table, fields, num_quest)

print sql_insert
print values
cursor.execute(sql_insert,values)

In [ ]:
director_list_106[3]

In [ ]:
print row.values()[1]

In [ ]:
for index, row in enumerate(director_list):
    # row data        
    fields = ','.join([ '['+e+ ']' for e in row.keys()]) # 欄位        
    values = row.values() # 值
    num_quest = '?' + ',?' * (len(row.keys())-1)
    try:
        sql_insert = u"""INSERT INTO {0}({1}) VALUES({2})""".format(
            table, fields, num_quest)

#         cursor.execute(sql_insert, values)
        # print "row index : %d, success!" %index
    except pypyodbc.IntegrityError as e:
        print 'Data key Duplicate!! Forbiden'

In [ ]:
print values[1]

In [ ]:
table = u'公司董監事'
print sql_insert
print(values)
cursor.execute(sql_insert,values)

In [ ]:
print u'\u6240\u4ee3\u8868\u6cd5\u4eba'